In [2]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
import os
# Set paths
biomed_path = "/content/drive/MyDrive/BIOMED"
ham10000_path = os.path.join(biomed_path, "HAM10000")

In [4]:
import os
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.inception_resnet_v2 import InceptionResNetV2, preprocess_input
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import regularizers


# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

# Set paths
biomed_path = "/content/drive/MyDrive/BIOMED"
ham10000_path = os.path.join(biomed_path, "HAM10000")

# Load metadata
metadata = pd.read_csv(os.path.join(biomed_path, "HAM10000_metadata.csv"))

# Update the image_id column to include the full path to the images
metadata["image_id"] = metadata.apply(lambda row: os.path.join(biomed_path, "HAM10000_images_part_1", row["image_id"] + ".jpg") if row["image_id"][0] == 'I' else os.path.join(biomed_path, "HAM10000_images_part_2", row["image_id"] + ".jpg"), axis=1)


# Split the dataset into training, validation, and test sets
train_df, test_df = train_test_split(metadata, test_size=0.2, random_state=42, stratify=metadata["dx"])
train_df, val_df = train_test_split(train_df, test_size=0.25, random_state=42, stratify=train_df["dx"])

# Data preprocessing and augmentation
train_datagen = ImageDataGenerator(
    preprocessing_function=preprocess_input,
    rotation_range=20,
    zoom_range=0.1,
    width_shift_range=0.1,
    height_shift_range=0.1,
    horizontal_flip=True,
    vertical_flip=True,
)

val_datagen = ImageDataGenerator(preprocessing_function=preprocess_input)

image_size = 299
batch_size = 32

train_generator = train_datagen.flow_from_dataframe(
    train_df,
    x_col="image_id",
    y_col="dx",
    target_size=(image_size, image_size),
    batch_size=batch_size,
    class_mode="categorical",
)

val_generator = val_datagen.flow_from_dataframe(
    val_df,
    x_col="image_id",
    y_col="dx",
    target_size=(image_size, image_size),
    batch_size=batch_size,
    class_mode="categorical",
)
# Load Inception-ResNet-v2 model
base_model = InceptionResNetV2(weights="imagenet", include_top=False, input_shape=(image_size, image_size, 3))

# Add custom layers
x = base_model.output
x = GlobalAveragePooling2D()(x)
output = Dense(7, activation="softmax", kernel_regularizer=regularizers.l2(0.01))(x)

model = Model(inputs=base_model.input, outputs=output)


# Freeze the base model layers
#for layer in base_model.layers:
 #   layer.trainable = False

# Compile the model
model.compile(optimizer=Adam(lr=0.001), loss="categorical_crossentropy", metrics=["accuracy"])

# Train the model
num_epochs = 25
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.n // batch_size,
    validation_data=val_generator,
    validation_steps=val_generator.n // batch_size,
    epochs=num_epochs,
)


# Save the trained model
model.save("inception_resnet_v2_ham10000.h5")


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Found 3042 validated image filenames belonging to 7 classes.


/usr/local/lib/python3.9/dist-packages/keras/preprocessing/image.py:1137: UserWarning: Found 2967 invalid image filename(s) in x_col="image_id". These filename(s) will be ignored.
  warnings.warn(


Found 968 validated image filenames belonging to 7 classes.


/usr/local/lib/python3.9/dist-packages/keras/preprocessing/image.py:1137: UserWarning: Found 1035 invalid image filename(s) in x_col="image_id". These filename(s) will be ignored.
  warnings.warn(


219055592/219055592 [==============================] - 10s 0us/step


Epoch 1/25
95/95 [==============================] - 2665s 27s/step - loss: 0.9201 - accuracy: 0.7319 - val_loss: 85.2443 - val_accuracy: 0.3958
Epoch 2/25
95/95 [==============================] - 146s 2s/step - loss: 0.6755 - accuracy: 0.7797 - val_loss: 3.7382 - val_accuracy: 0.7135
Epoch 3/25
95/95 [==============================] - 150s 2s/step - loss: 0.5893 - accuracy: 0.8040 - val_loss: 2.5213 - val_accuracy: 0.1854
Epoch 4/25
95/95 [==============================] - 146s 2s/step - loss: 0.5526 - accuracy: 0.8146 - val_loss: 1.1008 - val_accuracy: 0.6062
Epoch 5/25
95/95 [==============================] - 149s 2s/step - loss: 0.5234 - accuracy: 0.8133 - val_loss: 1.2173 - val_accuracy: 0.6417
Epoch 6/25
95/95 [==============================] - 149s 2s/step - loss: 0.5008 - accuracy: 0.8206 - val_loss: 1.1245 - val_accuracy: 0.7510
Epoch 7/25
95/95 [==============================] - 150s 2s/step - loss: 0.4404 - accuracy: 0.8502 - val_loss: 0.7164 - val_accuracy: 0.7677
Epoch 8/25

KeyboardInterrupt: ignored

In [5]:
import os

part1_path = os.path.join(ham10000_path, "HAM10000_images_part_1")
part2_path = os.path.join(ham10000_path, "HAM10000_images_part_2")

print("Files in HAM10000_images_part_1:")
print(os.listdir(part1_path)[:10])  # Print first 10 files

print("\nFiles in HAM10000_images_part_2:")
print(os.listdir(part2_path)[:10])  # Print first 10 files


Files in HAM10000_images_part_1:


FileNotFoundError: ignored